#MOVIE RECOMMENDATION USING RATINGS

In [0]:
import pandas as pd

In [0]:
from fuzzywuzzy import process

In [0]:
def movie_recommender(movie_name,df,Data,n):
    idx= process.extractOne(movie_name,df['title'])[2]
    print("movie selected: " ,df['title'][idx], "Index: ",idx)
    print("Movie recommendation......")
    Distance, indices = model.kneighbors(Data[idx],n_neighbors = n)

    rec_movies={"indices": [i for i in indices], "movies":[df['title'][i].where(i!=idx) for i in indices]}
    for i in indices:
        print(df['title'][i].where(i!=idx))
        # rec_movies.append(df['title'][i].where(i!=idx))
    dataframe=pd.DataFrame(rec_movies, columns = ['indices', 'movies'])
    return(dataframe)
    
    



In [0]:
from scipy.sparse import csr_matrix
import pandas as pd
import pyspark.pandas as ps
from pyspark.sql.functions import explode, col

# CREATING WIDGET TO TAKE INPUT FROM THE USER
dbutils.widgets.text(name="Entered_movie", defaultValue="Avatar", label="Type movie name")
input_movie = dbutils.widgets.get("Entered_movie")
# print(input_movie)

# READING DATA FROM THE CSV FILE IN THE DBFS AND CREATING ITS SPARK DATAFRAME
df = spark.read.format("csv")\
.option("inferSchema", "true") \
.option("header", "true") \
.option("sep", ",") \
.load("dbfs:/FileStore/tables/movies_y.csv").select("movieId", "title")

df_pandas = df.toPandas()

ratings_df = spark.read.format("csv")\
.option("inferSchema", "true") \
.option("header", "true") \
.option("sep", ",") \
.load("dbfs:/FileStore/tables/ratings.csv").select("userId", "movieId", "rating")

ratings_df_p = ratings_df.toPandas()

# CONVERTING OUR DATA TO TRAIN THE MODEL
ratings_table = ratings_df_p.pivot(index="movieId", columns="userId", values="rating").fillna(0)
mat_rating = csr_matrix(ratings_table.values)

# IMPORTING NearestNeighbors MODEL AND TRAINING OUR MODEL BY INSERTING OUR DATA IN IT 
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(metric= "cosine", algorithm="brute", n_neighbors=20)
model.fit(mat_rating)

# CALLING OUR CREATED FUNCTION TO GIVE THE OUTPUT USING TRAINED MODEL 
Recommended_mov= movie_recommender(input_movie, df_pandas, mat_rating, 10)

# CONVERTING OUR OUTPUT INTO SPARK DATAFRAME FROM PANDAS DATAFRAME TO DISPLAY ON TYHE DASHBOARD
Recommended_mov= Recommended_mov.fillna(0)
Recommended_mov['movies']=Recommended_mov['movies'].astype(str)

rec_spark = ps.DataFrame(Recommended_mov).to_spark()


movie selected:  Avatar (2009) Index:  7212
Movie recommendation......
7331    St Trinian's 2: The Legend of Fritton's Gold (...
9158                       A Very Murray Christmas (2015)
7842                                   John Carter (2012)
7583                     Daffy Duck's Quackbusters (1988)
9118                                    Witch Hunt (1999)
7917                                        Presto (2008)
8812                                 Power/Rangers (2015)
7922    Dragon Ball Z: The Return of Cooler (Doragon b...
3686                  Adventures of Huck Finn, The (1993)
9212                Requiem for the American Dream (2015)
Name: title, dtype: object


In [0]:
display(rec_spark)

indices,movies
"List(7331, 9158, 7842, 7583, 9118, 7917, 8812, 7922, 3686, 9212)","7331 St Trinian's 2: The Legend of Fritton's Gold (... 9158 A Very Murray Christmas (2015) 7842 John Carter (2012) 7583 Daffy Duck's Quackbusters (1988) 9118 Witch Hunt (1999) 7917 Presto (2008) 8812 Power/Rangers (2015) 7922 Dragon Ball Z: The Return of Cooler (Doragon b... 3686 Adventures of Huck Finn, The (1993) 9212 Requiem for the American Dream (2015) Name: title, dtype: object"


CREATING TABLES AND VISUALIZATION TO DISPLAY ON DASHBOARD

In [0]:
rec_spark.createOrReplaceTempView("temp_table")
rec_spark.write.mode("overwrite").saveAsTable("default.rec_mov")
ratings_df.createOrReplaceTempView("temp_table1")
ratings_df.write.mode("overwrite").saveAsTable("default.rating_table")
df = spark.read.format("csv")\
.option("inferSchema", "true") \
.option("header", "true") \
.option("sep", ",") \
.load("dbfs:/FileStore/tables/movies_y.csv").select("movieId", "title", "genres")
df.createOrReplaceTempView("temp_table2")
df.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("default.movie_table")




# rec_mov.withcolumn("indices", explode(col("indices")))

In [0]:
%sql
create or replace table new_movie as
select *, row_number() over (order by movieId) as index from movie_table

num_affected_rows,num_inserted_rows


In [0]:
%sql
create or replace table rec_ind as
select explode(indices) as ind from rec_mov;

num_affected_rows,num_inserted_rows


In [0]:
%sql
select  mt.title, avg(rt.rating) as overall_rating from rating_table rt join 
new_movie mt on rt.movieId = mt.movieId where mt.index - 1 in (select ind from rec_ind)
group by mt.title

title,overall_rating
"Adventures of Huck Finn, The (1993)",2.5
Power/Rangers (2015),4.0
A Very Murray Christmas (2015),2.5
Daffy Duck's Quackbusters (1988),2.0
John Carter (2012),3.090909090909091
Requiem for the American Dream (2015),2.5
St Trinian's 2: The Legend of Fritton's Gold (2009),2.0
Presto (2008),5.0
Dragon Ball Z: The Return of Cooler (Doragon bôru Z 6: Gekitotsu! Hyakuoku pawâ no senshi) (1992),4.0
Witch Hunt (1999),2.5


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
select  mt.title, mt.genres from rating_table rt join 
new_movie mt on rt.movieId = mt.movieId where mt.index - 1 in (select ind from rec_ind)
group by mt.title,mt.genres

title,genres
John Carter (2012),Action|Adventure|Sci-Fi|IMAX
"Adventures of Huck Finn, The (1993)",Adventure|Children|Comedy|Drama
Witch Hunt (1999),Crime|Drama
Daffy Duck's Quackbusters (1988),Animation|Children|Comedy|Horror
A Very Murray Christmas (2015),Comedy
Dragon Ball Z: The Return of Cooler (Doragon bôru Z 6: Gekitotsu! Hyakuoku pawâ no senshi) (1992),Action|Adventure|Animation
Power/Rangers (2015),Action|Adventure|Sci-Fi
Requiem for the American Dream (2015),Documentary
Presto (2008),Animation|Children|Comedy|Fantasy
St Trinian's 2: The Legend of Fritton's Gold (2009),Adventure|Comedy
